In [2]:
import tensorflow as tf

In [3]:
def read_data(file_queue):    #csv文件处理函数
    reader = tf.TextLineReader(skip_header_lines=1) # 每次读取一行
    key,value = reader.read(file_queue)
    
    defaults = [[0],[0.],[0.],[0.],[0.],[0]]  # 为每个字段设置初始值
    csvcolumn = tf.decode_csv(value,defaults)  # tf.decode_csv对每一行进行解析
    
    featurecolumn = [i for i in csvcolumn[1:-1]]  # 分离出列中的样本数据列
    labelcolumn = csvcolumn[-1]  # 分离出列中的标签数据列
    
    return tf.stack(featurecolumn),labelcolumn


In [4]:
def create_pipeline(filename,batch_size,num_epochs=None):  # 创建队列数据集函数
    # 创建一个输入队列
    file_queue = tf.train.string_input_producer([filename],num_epochs=num_epochs)
    
    feature,label = read_data(file_queue)  # 载入数据和标签
    
    min_after_dequeue = 1000 # 设置队列中的最少数据条数（取完之后保证还有1000条）
    capacity = min_after_dequeue + batch_size  # 队列的长度
    
    feature_batch,label_batch = tf.train.shuffle_batch(  # 生成乱序的批次数据
        [feature,label],batch_size=batch_size,capacity=capacity,
        min_after_dequeue=min_after_dequeue
    )
    return feature_batch,label_batch  # 返回指定批次的数据

In [5]:
# 读取训练集
x_train_batch, y_train_batch = create_pipeline('iris_training.csv',32,num_epochs=100)

# 读取测试集
x_test,y_test  = create_pipeline('iris_test.csv',32) 